In [1]:
import seqio
import tensorflow as tf
import functools


2023-10-30 15:03:56.270572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/pipenv/KLab_MultiModalModel-XfRlqCgI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentencepiece_model_file = "gs://t5-data/vocabs/cc_all.32000.100extra/sentencepiece.model"
vocab = seqio.SentencePieceVocabulary(sentencepiece_model_file)


In [3]:
# seqio.map_over_dataset is decorator to map decorated function 
# (e.g., sample_from_answers below) over all examples in a dataset.
# for details, please refer to seqio.map_over_dataset() documentation.
@seqio.map_over_dataset(num_seeds=1)
def sample_from_answers_task1(x, seed):
    x["inputs"] += ":task1"
    answers = x['targets']
    sample_id = tf.random.stateless_uniform([],
                                            seed=seed,
                                            minval=0,
                                            maxval=len(answers),
                                            dtype=tf.int32)
    x['targets'] = answers[sample_id]
    return x


In [4]:
seqio.TaskRegistry.remove('my_simple_task')
seqio.TaskRegistry.add(
    'my_simple_task',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
       sample_from_answers_task1,
       seqio.preprocessors.tokenize,
    ],
    output_features={
        'inputs': seqio.Feature(vocabulary=vocab),
        'targets': seqio.Feature(vocabulary=vocab),
    }
)


In [5]:
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())


2023-10-30 15:03:57.041530: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Instructions for updating:
Use `tf.data.Dataset.random(...)`.


2023-10-30 15:03:57.523187: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Instructions for updating:
Use `tf.data.Dataset.random(...)`.
2023-10-30 15:03:59.086640: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[{'inputs_pretokenized': b'who is the girl in more than you know:task1',
  'inputs': array([  113,    19,     8,  3202,    16,    72,   145,    25,   214,
            10, 23615,   536], dtype=int32),
  'targets_pretokenized': b'Romi Van Renterghem.',
  'targets': array([12583,    23,  4480,  9405,    49,   122,  6015,     5],
        dtype=int32)}]

In [6]:
# seqio.map_over_dataset is decorator to map decorated function 
# (e.g., sample_from_answers below) over all examples in a dataset.
# for details, please refer to seqio.map_over_dataset() documentation.
@seqio.map_over_dataset(num_seeds=1)
def sample_from_answers_task2(x, seed):
    x["inputs"] += ":task2"
    answers = x['targets']
    sample_id = tf.random.stateless_uniform([],
                                            seed=seed,
                                            minval=0,
                                            maxval=len(answers),
                                            dtype=tf.int32)
    x['targets'] = answers[sample_id]
    return x


In [7]:
seqio.TaskRegistry.remove('my_simple_task2')
seqio.TaskRegistry.add(
    'my_simple_task2',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
       sample_from_answers_task2,
       seqio.preprocessors.tokenize,
   ],
    output_features={
        'inputs': seqio.Feature(vocabulary=vocab),
        'targets': seqio.Feature(vocabulary=vocab),
    }
)


In [8]:
task = seqio.TaskRegistry.get('my_simple_task2')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())


2023-10-30 15:03:59.665668: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[{'inputs_pretokenized': b'who is the girl in more than you know:task2',
  'inputs': array([  113,    19,     8,  3202,    16,    72,   145,    25,   214,
            10, 23615,   357], dtype=int32),
  'targets_pretokenized': b'Romi Van Renterghem.',
  'targets': array([12583,    23,  4480,  9405,    49,   122,  6015,     5],
        dtype=int32)}]

In [9]:
seqio.MixtureRegistry.add(
    'my_simple_mixture',
    [('my_simple_task', 0.5), 'my_simple_task2'],
    default_rate=1.0
)


In [11]:
mixture = seqio.MixtureRegistry.get('my_simple_mixture')
ds = mixture.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(10).as_numpy_iterator())


2023-10-30 15:04:55.672006: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-10-30 15:04:55.672739: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[{'inputs': array([  113,    19,     8,  3202,    16,    72,   145,    25,   214,
            10, 23615,   357], dtype=int32),
  'targets': array([12583,    23,  4480,  9405,    49,   122,  6015,     5],
        dtype=int32)},
 {'inputs': array([  116,   405,   467,   147,   388,   369,    91,    30,  3134,
            89, 17591,    10, 23615,   357], dtype=int32),
  'targets': array([ 1332, 12992,   846], dtype=int32)},
 {'inputs': array([  116,    47,     8,     3,  5397,    15,    13,     8,  4913,
          8769,    52,  1545,    10, 23615,   357], dtype=int32),
  'targets': array([1003, 4327,  104, 3916], dtype=int32)},
 {'inputs': array([11003,    13,    16,  8603,  2717,    16,  1353,    13, 19680,
             3,   122,    26,   102,    10, 23615,   357], dtype=int32),
  'targets': array([17353,    18, 15599,     7,    17], dtype=int32)},
 {'inputs': array([  113,    47,     8,   163,   178,  2753,   113,     7, 18573,
            47,    92,  2753,    10, 23615,   357], dtype=i

In [18]:
task1_cout = 0
task2_cout = 0
total_num = 3000
for index,ex in zip(range(total_num),ds.as_numpy_iterator()):
    print(f'index: {index}')
    print(f'inputs: {vocab.decode(ex["inputs"])}')
    print(f'targets: {vocab.decode(ex["targets"])}')
    if 'task1' in vocab.decode(ex["inputs"]):
        task1_cout += 1
    elif 'task2' in vocab.decode(ex["inputs"]):
        task2_cout += 1
print(f'task1_cout: {task1_cout/total_num}')
print(f'task2_cout: {task2_cout/total_num}')


index: 0
inputs: who is the girl in more than you know:task2
targets: Romi Van Renterghem.
index: 1
inputs: when does game over man come out on netflix:task2
targets: March 23, 2018
index: 2
inputs: when was the rime of the ancient mariner written:task2
targets: 1797–98
index: 3
inputs: rank of indian economy in terms of nominal gdp:task2
targets: seventh-largest
index: 4
inputs: who was the only us president whos grandfather was also president:task2
targets: Benjamin Harrison
index: 5
inputs: who wrote i think i'm going back:task2
targets: Gerry Goffin and Carole King
index: 6
inputs: who is the girl in more than you know:task1
targets: Romi Van Renterghem.
index: 7
inputs: when does game over man come out on netflix:task1
targets: March 23, 2018
index: 8
inputs: when was the rime of the ancient mariner written:task1
targets: 1797–98
index: 9
inputs: who's the highest-paid basketball player in the nba:task2
targets: Stephen Curry
index: 10
inputs: when did the song sounds of silence c

2023-10-30 15:09:33.083926: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-10-30 15:09:33.084403: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
